The purpose of this notebook is to retrieve the sentences from which the dataset labels are extracted in the publications, as well as the name of the section where they are located.

In [1]:
import re

import json
import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm

In [2]:
DATA_PATH = '../input/challenge-data/train/'

In [3]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [6]:
train = pd.read_csv('../input/challenge-data/train.csv')
train

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
...,...,...,...,...,...
19656,b3498176-8832-4033-aea6-b5ea85ea04c4,RSNA International Trends: A Global Perspectiv...,RSNA International COVID-19 Open Radiology Dat...,RSNA International COVID Open Radiology Database,rsna international covid open radiology database
19657,f77eb51f-c3ac-420b-9586-cb187849c321,MCCS: a novel recognition pattern-based method...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...
19658,ab59bcdd-7b7c-4107-93f5-0ccaf749236c,Quantitative Structure–Activity Relationship M...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...
19659,fd23e7e0-a5d2-4f98-992d-9209c85153bb,A ligand-based computational drug repurposing ...,CAS COVID-19 antiviral candidate compounds dat...,CAS COVID-19 antiviral candidate compounds dat...,cas covid 19 antiviral candidate compounds dat...


In [23]:
locations = []
# lenteurs 14800, 16686
for id, row in tqdm(train.iterrows()):
    with open(DATA_PATH+row.Id+'.json') as f:
        f1 = json.load(f)
    for section in f1:
        sentences = nltk.tokenize.sent_tokenize(section['text'])
        for sentence in sentences:
            if row.dataset_label in sentence:
                locations.append([row.Id, section['section_title'], sentence, row.dataset_label])

19661it [05:34, 58.73it/s] 


In [8]:
locations = []
for id, row in tqdm(train.iterrows()):
    with open(DATA_PATH+row.Id+'.json') as f:
        f1 = json.load(f)
    for section in f1:
        sentences = nltk.tokenize.sent_tokenize(section['text'])
        for sentence in sentences:
            cleaned_sentence = clean_text(sentence)
            if row.cleaned_label in cleaned_sentence:
                locations.append([row.Id, section['section_title'], cleaned_sentence, row.cleaned_label])

19661it [07:26, 44.05it/s]


In [26]:
locations_df = pd.DataFrame(locations, columns = ['Id', 'section_title', 'sentence', 'dataset_label'])
len(locations_df)

54804

In [10]:
# /!\ si un dataset_label est une sous-chaîne d'un autre, alors les phrases récupérées pour le second seront aussi récupérées par le premier.

In [27]:
locations_df.loc[(locations_df.Id == 'eefeaeea-af30-4ba6-b23c-9f86a3f50482') & (locations_df.sentence == 'In contrast, the Baltimore Longitudinal Study of Aging (BLSA) found that in men, higher levels of testosterone were associated with a decrease in arterial stiffness over a 12-year followup.')]

,Id,section_title,sentence,dataset_label
1003,eefeaeea-af30-4ba6-b23c-9f86a3f50482,,"In contrast, the Baltimore Longitudinal Study ...",Baltimore Longitudinal Study of Aging (BLSA)
2840,eefeaeea-af30-4ba6-b23c-9f86a3f50482,,"In contrast, the Baltimore Longitudinal Study ...",Baltimore Longitudinal Study of Aging


Data not totally clean

In [106]:
#locations_df.loc[(locations_df.Id == 'dc23ed8c-f3ac-4c5b-a6a6-467a1a93a978') & (locations_df.section_title == 'B17FFEDACT') & (locations_df.sentence == 'SOURCE: U.S. Department of Education, National Center for Education Statistics, 2012/17 Beginning Postsecondary Students Longitudinal Study (BPS:12/17).')]

In [107]:
#locations_df.loc[(locations_df.Id == 'b8076a2f-4677-43fe-8dfa-c94de4a6be29') & (locations_df.section_title == 'Key facts:') & (locations_df.sentence == 'Source: National Education Longitudinal Study (NELS:88).')]

In [103]:
locations_df.groupby(['Id', 'section_title', 'sentence']).size().to_frame('count').sort_values('count')

count
Id                                   section_title                                      sentence                                                 
0007f880-0a9b-492d-9a58-76eb0b0e0bd7 Introduction                                       In fact, organizations are now identifying digi...      1
a323c88e-d24e-414d-84b9-4e70117ce7c5 Polygenic risk score (PRS) analyses on all outc... This situation can be expected to improve somew...      1
                                                                                        in ADNI), ours are the first to combine genomic...      1
                                     Sample and phenotype description                   Replication data used in the preparation of thi...      1
                                                                                        The ADNI was launched in 2003 as a public-priva...      1
...                                                                                                                                           ...
08a475b1-ed5d-4635-b358-7c91fc63e520 EDUCATIONAL AND LABOR MARKET OUTCOMES OF THE EL... NOTE: CTE = career and technical education; ELS...     18
9801d54f-6181-45fb-a49c-552402a3019f U.S. TIMSS and PIRLS 2011 Technical Report and ... SOURCE: International Association for the Evalu...     18
0606313f-1f86-4520-aac9-3cfecf20db11                                                    SOURCE: U.S. Department of Education, National ...     25
b8076a2f-4677-43fe-8dfa-c94de4a6be29 Key facts:                                         Source: National Education Longitudinal Study (...     26
dc23ed8c-f3ac-4c5b-a6a6-467a1a93a978 B17FFEDACT                                         SOURCE: U.S. Department of Education, National ...     32

[47972 rows x 1 columns]

Afin de supprimer les phrases récupérées en double lorsqu'un dataset_label est une sous-chaine d'un autre, on trie par longueur de dataset_label descendante et on supprime les doublons.  
/!\ risque de supprimer des phrases où le même dataset est référencé plusieurs fois, une première avec la longue dénomination et une seconde fois avec la courte dénomination.  
Éventuellent régler ça en amont, au moment de la récupération des phrases, d'autant plus que ça pourrait être intéressant de tester pour chaque publication toutes les dénominations des datasets qui y apparaissent plutôt que les dénominations reportées dans train.csv

In [28]:
locations_df['label_length'] = locations_df['dataset_label'].str.len()

In [29]:
locations_df.sort_values('label_length', ascending=False, inplace=True)

In [30]:
locations_df = locations_df[~locations_df.duplicated(['Id', 'sentence', 'section_title'])].reset_index(drop=True)

le doublon a bien été supprimé, et on a conservé le label le plus long:

In [117]:
locations_df.loc[(locations_df.Id == 'eefeaeea-af30-4ba6-b23c-9f86a3f50482')& (locations_df.sentence == 'In contrast, the Baltimore Longitudinal Study of Aging (BLSA) found that in men, higher levels of testosterone were associated with a decrease in arterial stiffness over a 12-year followup.')]

,Id,section_title,sentence,dataset_label,label_length
6261,eefeaeea-af30-4ba6-b23c-9f86a3f50482,,"In contrast, the Baltimore Longitudinal Study ...",Baltimore Longitudinal Study of Aging (BLSA),44


In [31]:
locations_df.sort_values('Id', inplace=True)
locations_df.reset_index(drop=True, inplace=True)
locations_df

,Id,section_title,sentence,dataset_label,label_length
0,0007f880-0a9b-492d-9a58-76eb0b0e0bd7,Introduction,"In fact, organizations are now identifying dig...",Program for the International Assessment of Ad...,62
1,0008656f-0ba2-4632-8602-3017b44c2e90,LITERATURE REVIEW,"International studies on student achievement, ...",Trends in International Mathematics and Scienc...,53
2,000e04d6-d6ef-442f-b070-4309493221ba,Example: Farm Income and Farm Household Wealth,The Agricultural Resources Management Survey (...,Agricultural Resources Management Survey,40
3,000e04d6-d6ef-442f-b070-4309493221ba,Highlights,1 manages access to results of the Agricultura...,Agricultural Resources Management Survey,40
4,000efc17-13d8-433d-8f62-a3932fe4f3b8,ABCC9 and cerebral blood flow,This was performed by measuring CBF in conveni...,ADNI,4
...,...,...,...,...,...
47967,ffd4d86a-0f26-44cc-baed-f0e209cc22af,II.1. MRI Brain Image database,Data used in the preparation of this article w...,Alzheimer's Disease Neuroimaging Initiative (A...,50
47968,ffe7f334-245a-4de7-b600-d7ff4e28bfca,Characterization of the SARS-CoV-2 virus,"Interestingly, the genome sequences of SARS-Co...",genome sequences of SARS-CoV-2,30
47969,ffeb3568-7aed-4dbe-b177-cbd7f46f34af,Polish research on the perception of mathematics,as part of the Program for International Stude...,Trends in International Mathematics and Scienc...,53
47970,ffee2676-a778-4521-b947-e1e420b126c5,,"My prior research illustrated, with use of Beg...",Beginning Postsecondary Student,31


In [33]:
# /!\ the text fields include commas
locations_df.to_csv('train_full-sentences.csv', sep='ç')

# -------------

In [123]:
print(sum(locations_df.duplicated('sentence')), sum(locations_df.duplicated(['sentence', 'section_title'])))

3581 994


In [126]:
locations_df.groupby('section_title').count().sort_values('Id', ascending=False)

,Id,sentence,dataset_label,label_length
section_title,,,,
Introduction,2940,2940,2940,2940
,2808,2808,2808,2808
Discussion,2576,2576,2576,2576
Abstract,2563,2563,2563,2563
Subjects,847,847,847,847
...,...,...,...,...
Case Study: U.S. Farm Household Adjustment to Income Shocks,1,1,1,1
"Law of Ukraine ""On Innovation Activity"" (2002)",1,1,1,1
Case studies: Changes in Arabian Sea,1,1,1,1


In [41]:
locations_df.groupby('sentence').count().sort_values('Id')

,Id,section_title
sentence,,
"\n\n\nMarketing contracts are common for specialty crops According to the USDA-ERS Agricultural Resource Management Survey (ARMS) in 2016, 39 percent of the value of production for fruits was under contract, and 37 percent of the value of production of vegetables was under contract Processed crops tend to be more heavily contracted than fresh.",1,1
"The National Hurricane Center (NHC) in conjunction with NOAA has divided the United States coasts into 38 elliptical basins for SLOSH modeling, each consisting of hundreds of grid cells.",1,1
The National Hurricane Center uses the SLOSH model in an operational mode to forecast hurricane surge elevations in a real-time mode.,1,1
"The National Institute of Aging has initiated the Alzheimer's Disease Neuroimaging Initiative (ADNI), a large observational study of patients with AD, patients with mild cognitive impairment (MCI), and cognitively normal volunteers to assess longitudinal changes in AD biomarkers.",1,1
"The National Institute of Aging, the National Institute of Biomedical Imaging and Bioengineering, the Food and Drug Administration, private pharmaceutical companies, and nonprofit organizations launched ADNI in 2003 as a publicprivate partnership.",1,1
...,...,...
Data used in the preparation of this article were obtained from the Alzheimer's Disease Neuroimaging Initiative (ADNI) database (adni.loni.usc.edu).,98,98
Subjects originally recruited for ADNI-1 and ADNI-GO had the option to be followed in ADNI-2.,110,110
"The ADNI was launched in 2003 as a public-private partnership, led by Principal Investigator Michael W. Weiner, MD.",125,125


In [21]:
locations_df['cleaned_section'] = locations_df['section_title'].apply(clean_text)

In [22]:
locations_df.groupby('cleaned_section').count().sort_values('Id', ascending=False)[:20]

,Id,section_title,sentence,dataset_label,label_length
cleaned_section,,,,,
introduction,3989,3989,3989,3989,3989
discussion,3378,3378,3378,3378,3378
,3024,3024,3024,3024,3024
abstract,2626,2626,2626,2626,2626
subjects,983,983,983,983,983
methods,949,949,949,949,949
participants,897,897,897,897,897
results,799,799,799,799,799
data,771,771,771,771,771


à comparer avec le nombre d'apparition de ces sections dans les fichiers source

Error on cleaning resulting in:  
30736,9762dd72-2656-4e1f-b158-7a139a2f6a76,"Strategic Design of the Antisense Oligonucleotide, ssDNA, Probes for


In [24]:
with open(DATA_PATH+'9762dd72-2656-4e1f-b158-7a139a2f6a76.json') as f:
    f1 = json.load(f)
f1